In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
# from sagemaker.image_uris import retrieve
from sagemaker.s3 import *
import sys

if int(sagemaker.__version__.split('.')[0]) == 2:
    !{sys.executable} -m pip install sagemaker==1.72.0
    print("Installing previous SageMaker Version. Please restart the kernel")
else:
    print("Version is good")

role = get_execution_role()
sess = sagemaker.Session(default_bucket=None)
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')


Version is good
Region = eu-central-1


In [18]:

# see https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html for which inputs to use
# see https://github.com/aws/deep-learning-containers/blob/master/available_images.md for registry paths with custom algorithms


training_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04"
print (training_image)

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04


In [19]:
!pip install sagemaker-experiments 
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [29]:
rawbucket= sess.default_bucket() # Alternatively you can use our custom bucket here. 
data_bucket = "treetracker-training-images"
prefix = 'sagemaker-modelmonitor' # use this prefix to store all files pertaining to this workshop.

dataprefix = prefix + '/data'
traindataprefix = prefix + '/train_data'
testdataprefix = prefix + '/test_data'
testdatanolabelprefix = prefix + '/test_data_no_label'
trainheaderprefix = prefix + '/train_headers'

s3_train_key = "imnet"
s3_validation_key = ""
s3_train = 's3://{}/{}/'.format(data_bucket, s3_train_key)
s3_validation = 's3://{}/{}/'.format(data_bucket, s3_validation_key)
sagemaker_train = 's3://{}/{}/'.format(rawbucket, s3_train_key)
sagemaker_validation = 's3://{}/{}/'.format(rawbucket, s3_validation_key)


In [30]:
train_data_location = rawbucket + '/' + traindataprefix
test_data_location = rawbucket+'/'+testdataprefix
print("Training data location = {}".format(train_data_location))
print("Test data location = {}".format(test_data_location))

Training data location = sagemaker-eu-central-1-053061259712/sagemaker-modelmonitor/train_data
Test data location = sagemaker-eu-central-1-053061259712/sagemaker-modelmonitor/test_data


In [34]:
# TODO: Figure out preprocessing instance jobs
from sagemaker.processing import ProcessingInput, ProcessingOutput

pre_input = ProcessingInput(source=s3_train, destination=None, input_name="pre")
pre_output = ProcessingOutput(source=None, destination=sagemaker_train)



In [35]:
from sagemaker.pytorch import PyTorch

# This is where you can add hyperparameters, framework used, point to the script, and define instances you want to train on. 
# ALl of this information is represented as environment variables passed to the instance. In your script, you can refer to these variables or 
# the argument. 
estimator = PyTorch(entry_point='mobilenet_v2.py',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo',
                        'train_split': 0.7
                    })

In [36]:
estimator.fit({"training": s3_train})

INFO:sagemaker:Creating training-job with name: pytorch-training-2020-12-15-04-32-32-421


2020-12-15 04:32:32 Starting - Starting the training job...
2020-12-15 04:32:34 Starting - Launching requested ML instances......
2020-12-15 04:33:39 Starting - Preparing the instances for training......
2020-12-15 04:34:52 Downloading - Downloading input data....................................
2020-12-15 04:40:55 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-15 04:40:52,594 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-12-15 04:40:52,598 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-12-15 04:40:52,613 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-15 04:40:57,736 sagemaker-containers INFO     Imported framew

UnexpectedStatusException: Error for Training job pytorch-training-2020-12-15-04-32-32-421: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python mobilenet_v2.py --backend gloo --epochs 6 --train_split 0.7"
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth
  0%|          | 0.00/13.6M [00:00<?, ?B/s] 82%|âââââââââ | 11.1M/13.6M [00:00<00:00, 117MB/s]100%|ââââââââââ| 13.6M/13.6M [00:00<00:00, 114MB/s]
Traceback (most recent call last):
  File "mobilenet_v2.py", line 549, in <module>
    trainer.train(args)
  File "mobilenet_v2.py", line 352, in train
    loss = loss_spec(is_tree_labels, is_tree_preds, box_labels, box_preds)
  File "mobilenet_v2.py", line 220, in loss_spec
    binary_detection_error = nn.BCEWithLogitsLoss(binary_preds.squeeze(), binary_label)# output, target
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/